# Querying Azure Sentinel via REST API using a Device Code

<table align="left" style="margin-left:25px">
  <tr>
    <th>Type</th>
    <th>Title</th>
    <th>Description</th>
  </tr>
  <tr>
      <td>Documentation</td>
      <td><b><a href="https://techcommunity.microsoft.com/t5/azure-sentinel/extending-azure-sentinel-apis-integration-and-management/ba-p/1116885">Extending Azure Sentinel: APIs, Integration and management automation</a></td>
    <td>Its highly recommended to follow this guide to get familiar with navigating and using Jupyter Notebooks. </td>
  </tr>
  <tr>
      <td>Tech Community</td>
    <td><b><a href="../../PowerShell/Getting-Started-README.ipynb">Access Azure Sentinel Log Analytics via API (Part 1)</a></td>
    <td>@rinure wrote a good article explaining how to query the API. Use this as a reference to understand how to query the Azure Sentinel API. </td>
  </tr>
</table> 

## Set your variables here (Uncomment)

In [16]:
<#
$config = @{
    tenantDomain = "domain.com"
    appID = "xxxxxx-23af-4ae5-a6ed-xxxxxx"                   ## Usually known as Client ID or Application ID
    sentinelWorkspaceID = "a89xxxxxxxx-4955-9f94xxxxxx900"
}
#>

## Import credential from storage path or Export a new XML file using above filled in parameters

In [ ]:
## Checks to see if config file exist and if not export credential file from above.
$storagePath = ".\AzSentinel.credential"
$config = (Import-CliXml -Path $storagePath)
if (!$config) {
    $config | Export-CliXml -Path $storagePath
} else {
    Write-Host -Foregroundcolor green "`nCredentials detected"
}

$resource = "https://api.loganalytics.io"
$authUrl = "https://login.microsoftonline.com/$($config.tenantDomain)";
$appID = $config.appID

Write-Host -ForeGroundColor Yellow "Authentication Parameters"
Write-Host "Resource URL: $($resource)"
Write-Host "Client APP ID: $($config.appId -replace "\d","*")"
Write-Host "Tenant Domain: $($config.tenantDomain)"

## Import Graph Authentication module to authenticate by Device Code
##### We will be importing the Device Code Authentication Module. To learn about this module go to notebook [here](https://demoloaderpowershellnotebook-skyserver.msappproxy.net/lab/tree/Graph-API/Authentication%20-%20Rest%20API%20-%20Device%20Code.ipynb)

In [ ]:
Import-Module "..\..\Graph-API\modules\GraphAuthentication.psm1" -Force  ## Imports the module
#Get-Command -Module "GraphAuthentication"                  ## Shows what commands were imported

$tokenResponse = Invoke-GraphAuthenticateDeviceCode -clientAppId $config.appID -tenantDomain $config.tenantDomain -Resource $resource

### Put your Azure Sentinel Hunting Query here

In [ ]:
## Enter your query here
$query = @"

SecurityAlert

"@

$timeSpan = "" ## You can leave blank or specify a date. But something HAS to be there. 
 
## This is the query converted to JSON format
$body = (@{
   query = $Query
   timespan = $timeSpan  
} | ConvertTO-JSON)
Write-Host -ForeGround Yellow "`nQuery"; $Query
Write-Host -ForeGround Yellow "JSON"; $body

## We will query the API

In [5]:
$URI = "https://api.loganalytics.io/v1/workspaces/$($config.sentinelWorkspaceID)/query"
$header = @{ 
    'Content-Type' = 'application/json'
    Accept = 'application/json'
    Authorization = "Bearer $($tokenResponse.access_token)" 
}

Write-Host "URI: $($URI -replace "[0-9]","*")"
$URI
$header

URI: https://api.loganalytics.io/v*/workspaces/a*******-*bde-****-*f**-*c**a**cc***/query
https://api.loganalytics.io/v1/workspaces/a8960181-0bde-4955-9f94-2c12a36cc900/query

Name                           Value
----                           -----
Accept                         application/json
Content-Type                   application/json
Authorization                  Bearer eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IlNzWnNCTmhaY0Yz…



## query Graph API and insert into variable

In [ ]:
$Result = (Invoke-RestMethod -Method POST -Uri $URI -Headers $header -body $body -ErrorAction Stop)
$Result.tables.rows | Select -First 1

## Grab certain element. 

In [15]:
$Result.tables.rows[0]

$Result.tables.rows[0][3]

a8960181-0bde-4955-9f94-2c12a36cc900

Monday, May 25, 2020 7:42:54 PM
DLP Policy - Sharepoint - Sensitive files shared externally
DLP Policy - Sharepoint - Sensitive files shared externally
Medium
Activity policy 'DLP Policy - Sharepoint - Sensitive files shared externally' was triggered by 'Bob Smith (bob@jingtoso.com)'
MCAS
Microsoft
073A5B74-5D2B-3E70-BBCE-B1CE0EE5EC2E
dee5f39b-e1fe-ad36-dc38-aec97082cf8e


MCAS_ALERT_CABINET_EVENT_MATCH_AUDIT
Unknown
False
Monday, May 25, 2020 7:42:49 PM
Monday, May 25, 2020 7:42:49 PM
Monday, May 25, 2020 7:42:54 PM

{
  "Cloud Applications": "Microsoft SharePoint Online"
}
[
  {
    "$id": "3",
    "Name": "bob",
    "UPNSuffix": "jingtoso.com",
    "AadUserId": "6836ed6d-2a15-4889-ad24-fbda46b5ffda",
    "Type": "account"
  },
  {
    "$id": "4",
    "AppId": 20892,
    "Name": "Microsoft SharePoint Online",
    "InstanceName": "Microsoft SharePoint Online",
    "Type": "cloud-application"
  }
]
Detection


[
  {
    "Href": "https://skyserver.p